In [1]:
import sys
import os.path as osp
import os
import numpy as np
from numpy.random import RandomState
import random
import torch
import torch_geometric
from torch_geometric.data import Data, DataLoader
import torch_geometric.transforms as T
from torch_geometric.explain import Explainer, GNNExplainer, PGExplainer
from torch_geometric.nn import GCNConv, SAGEConv, ChebConv, GatedGraphConv, Linear, global_mean_pool, global_max_pool
from torch.nn import functional as F
import pandas as pd
from tqdm import tqdm
import sklearn.metrics
import optuna
import torch.optim as optim
from GraphBetaExplainer import BetaExplainer

In [2]:
def set_seed(seed: int = 42) -> None:
    '''This function allows us to set the seed for the notebook across different seeds.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

In [3]:
set_seed(200)

Random seed set as 200


In [4]:
def get_data(num):
    '''This function allows us to load our data. We use the supergraph data - the original graph plus
        extra points associated with differnet points that are highly correlated to see if our explainers
        capture the ground truth data well'''
    labels = np.load(f'Time Experiments/sergio data/SERGIOsimu_{num}Sparse_noLibEff_cTypes.npy')
    features = np.load(f'Time Experiments/sergio data/SERGIOsimu_{num}Sparse_noLibEff_concatShuffled.npy')
    num_features = features.shape[1]
    num_classes = len(np.unique(labels))
    adj = np.load(f'Time Experiments/sergio data/ExtraPointsSergio{num}.npy')
    return adj, features, labels, num_features, num_classes

In [5]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, data, output_size):
        super(GCN, self).__init__()
        self.conv1 = SAGEConv(1, hidden_channels)
        self.embedding_size = hidden_channels
    def forward(self, x, edge_index, batch=None, edge_weights=None):
        if batch is None: # No batch given
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            batch = torch.zeros(x.size(0), dtype=torch.long).to(device)
        x = self.conv1(x, edge_index, edge_weights)
        x = F.dropout(x, p=0.2, training=self.training)
        x = global_max_pool(x, batch)
        return x

In [6]:
def train(model, train_loader):
    model.train()
    avgLoss = 0
    for data in tqdm(train_loader, total=47):  # Iterate in batches over the training dataset.
        data.x = torch.reshape(data.x, (data.x.shape[0], 1))
        data.x = data.x.type(torch.FloatTensor)
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)# Perform a single forward pass
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
        avgLoss += loss
    return avgLoss / 47

In [7]:
def test(model, loader):
    model.eval()
    correct = 0
    avgAUC = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        data.x = torch.reshape(data.x, (data.x.shape[0], 1))
        data.x = data.x.type(torch.FloatTensor)
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)

In [8]:
sparse = 50
adj, features, labels, num_features, num_classes = get_data(sparse)
edge_index = torch.tensor(adj, dtype=torch.int64)
ei = edge_index
features = features.astype(np.float32)
sz = np.array(adj).shape[1]
num_edges = sz
edge_weight = torch.ones(sz)
num_graphs = len(labels)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
graph_data = []
shuffle_index = []
for i in range(0, num_graphs):
    shuffle_index.append(i)
shuffle_index = np.array(random.sample(shuffle_index, num_graphs))
shuffle_index = shuffle_index.astype(np.int32)
num_train = int(len(shuffle_index)* 0.8)
num_test = num_graphs - num_train
train_dataset = []
test_dataset = []
for j in range(0, num_graphs):
    i = shuffle_index[j]
    x = torch.tensor(features[i])
    y = torch.tensor(labels[i])
    data = Data(x=x, y=y, edge_index=edge_index)
    graph_data.append(data)
    if j < num_train:
        train_dataset.append(data)
    else:
        test_dataset.append(data)
y = torch.tensor(labels)
graph_data = torch_geometric.data.Batch.from_data_list(graph_data)
graph_data = DataLoader(graph_data, batch_size=num_graphs)
print(graph_data)
dataset = graph_data
train_dataset = torch_geometric.data.Batch.from_data_list(train_dataset)
test_dataset = torch_geometric.data.Batch.from_data_list(test_dataset)

feat = torch.tensor(features)
adjacency = torch.tensor(adj, dtype=torch.int64)
y = torch.tensor(labels)

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

train_loader = DataLoader(train_dataset, batch_size=num_train, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=num_test, shuffle=False)
    
model = GCN(hidden_channels=2, data=dataset, output_size=num_classes).to(device)
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

C:\Users\whitn\anaconda3\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Number of training graphs: 1600
Number of test graphs: 400
GCN(
  (conv1): SAGEConv(1, 2, aggr=mean)
)


In [9]:
for epoch in range(1, 51):
    loss = train(model, train_loader)
    train_acc = test(model, train_loader)
    test_acc = test(model, test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

  2%|▏         | 1/47 [00:01<00:50,  1.09s/it]


Epoch: 001, Train Acc: 0.7494, Test Acc: 0.7450, Loss: 0.0146


  2%|▏         | 1/47 [00:03<02:25,  3.16s/it]


Epoch: 002, Train Acc: 0.7575, Test Acc: 0.7550, Loss: 0.0138


  2%|▏         | 1/47 [00:01<01:24,  1.84s/it]


...

Epoch: 048, Train Acc: 0.7788, Test Acc: 0.7650, Loss: 0.0125


  2%|▏         | 1/47 [00:01<01:20,  1.75s/it]


Epoch: 049, Train Acc: 0.7788, Test Acc: 0.7650, Loss: 0.0128


  2%|▏         | 1/47 [00:01<00:50,  1.10s/it]


Epoch: 050, Train Acc: 0.7788, Test Acc: 0.7700, Loss: 0.0126


In [10]:
print(sparse)
X = torch.tensor(features)
y = torch.tensor(labels)
# X = torch.reshape(X, (X.shape[0], X.shape[1], 1))
G = np.load(f'Time Experiments/sergio data/ExtraPointsSergio{sparse}.npy')
G = torch.tensor(G, dtype=torch.int64)
i = X.shape[1]

loader = graph_data
gt = np.load('Time Experiments/sergio data/gt_adj.npy')
lst1 = gt[0]
lst2 = gt[1]
gt = set()
for i in range(0, len(lst1)):
    pt = (lst1[i], lst2[i])
    gt.add(pt)
df_extra = np.load(f'Time Experiments/sergio data/ExtraPointsSergio{sparse}.npy')
lst1 = df_extra[0]
lst2 = df_extra[1]
gt_grn = [] # Initialize list denoting whether edges in supergraph are in the original graph
full_set = set()
sz = len(lst1)
for i in range(0, len(lst1)):
    pt = (lst1[i], lst2[i])
    full_set.add(pt)
    if pt in gt:
        gt_grn.append(1) # If in ground truth graph, add 1
    else:
        gt_grn.append(0) # Else add 0
groundtruth_mask = torch.tensor(gt_grn)
gt_grn = groundtruth_mask
    
false_negative_base = 0
l1 = np.load('Time Experiments/sergio data/gt_adj.npy')[0]
l2 = np.load('Time Experiments/sergio data/gt_adj.npy')[1]
for i in range(0, len(l1)):
    if (l1[i], l2[i]) not in full_set:
        false_negative_base += 1
print(f'Number of Data-based FNs: {false_negative_base}')
def faithfulness(model, X, G, edge_mask):
    org_vec = []
    for data in X:
        data.x = torch.reshape(data.x, (data.x.shape[0], 1))
        data.x = data.x.type(torch.FloatTensor)
        data = data.to(device)
        org_vec1 = model(data.x, G, data.batch).tolist()
        org_vec.append(org_vec1)
    org_vec = torch.tensor(org_vec)
    lst = []
    for i in range(0, edge_mask.shape[0]):
        if edge_mask[i] >= 0.5:
            lst.append(i)
    g = G[:, lst]
    
    pert_vec = []
    for data in X:
        data.x = torch.reshape(data.x, (data.x.shape[0], 1))
        data.x = data.x.type(torch.FloatTensor)
        data = data.to(device)
        pert_vec1 = model(data.x, g, data.batch).tolist()
        pert_vec.append(pert_vec1)
    pert_vec = torch.tensor(pert_vec)
    
    org_softmax = F.softmax(org_vec, dim=-1)
    pert_softmax = F.softmax(pert_vec, dim=-1)
    res = 1 - torch.exp(-F.kl_div(org_softmax.log(), pert_softmax, None, None, 'sum')).item()

    return res

50
Number of Data-based FNs: 109


In [11]:
def metrics(gt_grn, prediction_mask, false_negative_base):
    tp = 0
    tn = 0
    fn = false_negative_base
    fp = 0
    for ct in range(0, sz):
        if gt_grn[ct] == 1 and prediction_mask[ct] > 0.5:
            tp += 1
        elif gt_grn[ct] == 1 and prediction_mask[ct] <= 0.5:
            fn += 1
        elif gt_grn[ct] == 0 and prediction_mask[ct] <= 0.5:
            tn += 1
        elif gt_grn[ct] == 0 and prediction_mask[ct] > 0.5:
            fp += 1
        else:
            continue
    acc = (tp + tn) / (tp + tn + fn + fp)
    if tp + fp != 0:
        prec = tp / (tp + fp)
    else:
        prec = 0
    if tp + fp != 0:
        rec = tp / (tp + fn)
    else:
        rec = 0
    if prec != 0 and rec != 0:
        f1 = (2 * prec * rec) / (prec + rec)
    else:
        f1 = 0
    return acc, f1

In [12]:
def objective(trial):
    lr = trial.suggest_float('lrs', 1e-6, 0.2)
    alpha = trial.suggest_float('a', 0.1, 1)
    beta = trial.suggest_float('b', 0.1, 1)
    explainer = BetaExplainer(model, graph_data, G, torch.device('cpu'), 2000, alpha, beta)
    explainer.train(25, lr)
    prediction_mask = explainer.edge_mask()
    em = prediction_mask
    acc, f1 = metrics(groundtruth_mask, prediction_mask, 0)
    return acc

In [2]:
pruner = optuna.pruners.MedianPruner()
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(), pruner=pruner)
study.optimize(objective, n_trials=50)
print('Best hyperparameters:', study.best_params)
print('Best accuracy:', study.best_value)